In [ ]:
import os 
import time 
import json,math,sys,re
import pandas as pd 
from sklearn.metrics import accuracy_score, precision_score, f1_score, cohen_kappa_score,recall_score
import seaborn as sns
import matplotlib.pyplot as plt

import sys 
sys.path.append('../')

bible_info = {}
bible = {}
special = []
for bname in ['AKJV','ODRV','Geneva', 'Vulgate', 'Douay-Rheims', 'Tyndale', 'Wycliffe']:
    data = pd.read_csv(f"../assets/Bibles/{bname}.csv",header=None) 
    data = data.to_dict(orient="records")
    for entry in data: 
        key = entry[0]
        text = entry[6]
        # if re.search("[^\d\w\s\,\.\:\;\?\&\(\)\-\!\]\[\/\']+",text): 
        #     special.extend(re.findall("[^\d\w\s\,\.\:\;\?\&\(\)\-\!\]\[\/\']+",text))
        if re.search("Douay-Rheims",key): 
            if re.sub("Douay-Rheims","ODRV",key) in bible_info: 
                continue 
        if len(text.split(" ")) < 200: 
            bible_info[key] = (entry[1], entry[2], entry[3],text) # version, part, book,text
            if text not in bible: 
                bible[text] = []
            bible[text].append(key) 
        
        parts = re.split(r'(?<=[\.\?\!\:\;])',text)
        parts = [re.sub(r'\s+', ' ', p).strip() for p in parts if len(p.strip(" ")) > 0]
        if len(parts) > 1:
            for pidx, p in enumerate(parts):
                if p not in bible:
                    bible[p] = []
                bible[p].append(f"{key} - {pidx}")
                bible_info[f"{key} - {pidx}"] = (entry[1], entry[2], entry[3],p)

DS V3 - 40 mins for 597 entries 
GPT 4o - 7 mins for 597 entries 
Qwen 2.5-7B - 4 mins for 597 entries 
Qwen 2.5-14B
Llama-3.1 

Qwen 2.5-32B -
Llama-3.3-70B

# Add AKJV and ORDV to Old QP Data

In [106]:
inputfname = "Vul_notPriorInCited"
# inputfname = "preE_Vul"
# inputfname = "preE_qp"

data = pd.read_csv(f"../../QP/OLD_{inputfname}.csv")
data = data.to_dict(orient="records")

new_data = []
for item in data:
    seen = {} 
    if re.search(r"\'",item['verse_id']):
        verse_ids = re.findall("\'(.*?)\'",item['verse_id'])
    else: 
        verse_ids = [item['verse_id'].strip()]
    for v in verse_ids: 
        if "KJV" in v: 
            v = re.sub("KJV","AKJV",v)
        elif re.search("Douay",v): 
            new_v = re.sub("Douay-Rheims","ODRV",v)
            if new_v in bible_info: 
                v = new_v
        if v in bible_info: 
            text = bible_info[v][3]
            if text not in seen: 
                seen[text] = []
            if v not in seen[text]:
                seen[text].append(v)
        elif "-" in v: 
            orig_v = v 
            idx = int(v.split(" - ")[-1])
            v_id = v.split(" - ")[0]
            if idx <= 1: 
                v = v_id 
            else: 
                v = f"{v_id} - {idx-1}"
            if re.search("Douay",v): 
                new_v = re.sub("Douay-Rheims","ODRV",v)
                if new_v in bible_info: 
                    v = new_v
            if v in bible_info: 
                text = bible_info[v][3]
                if text not in seen: 
                    seen[text] = []
                if v not in seen[text]:
                    seen[text].append(v)
            else: 
                print(v,orig_v)
    for text, v_id in seen.items(): 
        new_item = item.copy()
        new_item['verse_text'] = text
        new_item['verse_id'] = "; ".join(v_id)
        new_data.append(new_item)
new_data = pd.DataFrame(new_data)
new_data = new_data.drop(columns=['score']) # 
# new_data = new_data.drop(columns=['Unnamed: 0']) # 
# new_data = new_data.drop(columns=['tcpID','sidx','loc','part','group']) # 
new_data = new_data.drop_duplicates()
new_data.to_csv(f"../../QP/{inputfname}.csv")
new_data

,verse_id,text,verse_text,label
0,Haggai 2.15 (Geneva) - 1,"Our callings, our labours, our actions, and th...","and so are all the workes of their hands, and ...",False
1,Mark 7.4 (Geneva) - 1,"Our callings, our labours, our actions, and th...","and many other things there be, which they hau...",False
2,Mark 7.4 (Geneva),"Our callings, our labours, our actions, and th...","And when they come from the market, except the...",False
3,2 Corinthians 5.9 (Geneva),"Our callings, our labours, our actions, and th...","Wherefore also we couet, that both dwelling at...",False
4,Psalms 127.2 (ODRV),"Our callings, our labours, our actions, and th...",Because thou shalt eate the labours of thy han...,True
...,...,...,...,...
613,Galatians 5.17 (ODRV) - 0,because the flesh euer lusteth against the Spirit,"For the flesh lusteth against the spirit, and ...",True
614,Galatians 5.19 (Tyndale),Manifesta sunt opera carnis &c. The dedes of t...,The dedes of the flesshe are manyfest whiche a...,True
615,Galatians 5.19 (Geneva),Manifesta sunt opera carnis &c. The dedes of t...,"Moreouer the workes of the flesh are manifest,...",True
616,Galatians 5.19 (Vulgate),Manifesta sunt opera carnis &c. The dedes of t...,"Manifesta sunt autem opera carnis, quæ sunt fo...",True


In [ ]:
inputfname = "Vul_notPriorInCited"
# inputfname = "preE_Vul"
# inputfname = "preE_qp"

data = pd.read_csv(f"../../QP/OLD_{inputfname}.csv")
data = data.to_dict(orient="records")

new_data = []
for item in data:
    seen = {} 
    if re.search(r"\'",item['verse_id']):
        verse_ids = re.findall("\'(.*?)\'",item['verse_id'])
    else: 
        verse_ids = [item['verse_id'].strip()]
    for v in verse_ids: 
        if "KJV" in v: 
            v = re.sub("KJV","AKJV",v)
        elif re.search("Douay",v): 
            new_v = re.sub("Douay-Rheims","ODRV",v)
            if new_v in bible_info: 
                v = new_v
        if v in bible_info: 
            text = bible_info[v][3]
            if text not in seen: 
                seen[text] = []
            if v not in seen[text]:
                seen[text].append(v)
        elif "-" in v: 
            orig_v = v 
            idx = int(v.split(" - ")[-1])
            v_id = v.split(" - ")[0]
            if idx <= 1: 
                v = v_id 
            else: 
                v = f"{v_id} - {idx-1}"
            if re.search("Douay",v): 
                new_v = re.sub("Douay-Rheims","ODRV",v)
                if new_v in bible_info: 
                    v = new_v
            if v in bible_info: 
                text = bible_info[v][3]
                if text not in seen: 
                    seen[text] = []
                if v not in seen[text]:
                    seen[text].append(v)
            else: 
                print(v,orig_v)
    for text, v_id in seen.items(): 
        new_item = item.copy()
        new_item['verse_text'] = text
        new_item['verse_id'] = "; ".join(v_id)
        new_data.append(new_item)
new_data = pd.DataFrame(new_data)
new_data = new_data.drop(columns=['score']) # 
# new_data = new_data.drop(columns=['Unnamed: 0']) # 
# new_data = new_data.drop(columns=['tcpID','sidx','loc','part','group']) # 
new_data = new_data.drop_duplicates()
new_data.to_csv(f"../../QP/{inputfname}.csv")
new_data

,verse_id,text,verse_text,label
0,Haggai 2.15 (Geneva) - 1,"Our callings, our labours, our actions, and th...","and so are all the workes of their hands, and ...",False
1,Mark 7.4 (Geneva) - 1,"Our callings, our labours, our actions, and th...","and many other things there be, which they hau...",False
2,Mark 7.4 (Geneva),"Our callings, our labours, our actions, and th...","And when they come from the market, except the...",False
3,2 Corinthians 5.9 (Geneva),"Our callings, our labours, our actions, and th...","Wherefore also we couet, that both dwelling at...",False
4,Psalms 127.2 (ODRV),"Our callings, our labours, our actions, and th...",Because thou shalt eate the labours of thy han...,True
...,...,...,...,...
613,Galatians 5.17 (ODRV) - 0,because the flesh euer lusteth against the Spirit,"For the flesh lusteth against the spirit, and ...",True
614,Galatians 5.19 (Tyndale),Manifesta sunt opera carnis &c. The dedes of t...,The dedes of the flesshe are manyfest whiche a...,True
615,Galatians 5.19 (Geneva),Manifesta sunt opera carnis &c. The dedes of t...,"Moreouer the workes of the flesh are manifest,...",True
616,Galatians 5.19 (Vulgate),Manifesta sunt opera carnis &c. The dedes of t...,"Manifesta sunt autem opera carnis, quæ sunt fo...",True


In [208]:
for v_id in bible_info: 
    if "1 Samuel 25.25 " in v_id:
        if " - " in v_id: continue 
        # if " - 0" not in v_id: continue   
        print(f'{v_id}\t\t\"{bible_info[v_id][3]}\"')

1 Samuel 25.25 (AKJV)		"Let not my lord, I pray thee, regard this man of Belial, euen Nabal: for as his name is, so is he: Nabal is his name, and folly is with him: But I thine handmaid saw not the yong men of my lord, whom thou didst send."
1 Samuel 25.25 (Geneva)		"Let not my lorde, I pray thee, regard this wicked man Nabal: for as his name is, so is he: Nabal is his name, and follie is with him: but I thine handmayde sawe not the yong men of my lord whom thou sentest."
1 Samuel 25.25 (Vulgate)		"Ne ponat, oro, dominus meus rex cor suum super virum istum iniquum Nabal: quoniam secundum nomen suum stultus est, et stultitia est cum eo: ego autem ancilla tua non vidi pueros tuos, domine mi, quos misisti."
1 Samuel 25.25 (Douay-Rheims)		"Let not my lord the king, I pray, regard this naughty man Nabal: for according to his name, he is a fool, and folly is with him: but I thy handmaid did not see thy servants, my lord, whom thou sentest."


# Read data

In [125]:
def read_data_and_labels(inputfname,m_to_l): 
    data = pd.read_csv(f"../../QP/{inputfname}.csv")
    data = data.rename(columns={'Unnamed: 0':'index'})
    for model in m_to_l:
        with open(f"../assets/QP_labels/{inputfname}_{model}.json","r") as f: 
            m_to_l[model] = json.load(f)
    return data, m_to_l

dataset_name = "Vul_notPriorInCited"
model_to_labels = {"DSV3":[],"GPT4o":[],
                   "Qwen_unsloth-Qwen2.5-7B":[],
                   "Qwen_unsloth-Qwen2.5-14B":[],
                   "Qwen_unsloth-Meta-Llama-3.1-8B":[]}

model_to_labels = {"Qwen_unsloth-Qwen2.5-7B_P1":[],
                   "Qwen_unsloth-Qwen2.5-7B_P2":[],
                   "Qwen_unsloth-Qwen2.5-7B_P3":[]}
annotations, model_to_labels = read_data_and_labels(dataset_name,model_to_labels)

len(annotations)

608

In [126]:
annotations = annotations.to_dict(orient='records')

In [127]:
ground = []
verses = []
model_to_pred = {x:[] for x in model_to_labels.keys()}

def new_label(label):
    if str(label).capitalize() == 'True': 
        return 1 
    else: 
        return 0 

for item in annotations: 
    label = item['label']
    index = str(item['index'])
    skip = False 
    for model, labels in model_to_labels.items(): 
        if index not in labels: 
            skip = True 
    if skip: continue
    ground.append(new_label(label))
    for model, labels in model_to_labels.items(): 
        model_to_pred[model].append(new_label(labels[index]))


# Compute evaluation stats 

In [129]:
scores = {
    "Model": [],
    "Accuracy": [],
    "Recall":[],
    "Precision": [],
    "F1_Score": [],
    "Cohen's_Kappa":[]
}

In [130]:
for pred_name, pred_labels in model_to_pred.items():
    if len(ground) == len(pred_labels):  
        pred_name = pred_name.split("_")[-1]
        Accuracy = round(accuracy_score(ground, pred_labels),4)
        Recall = round(recall_score(ground, pred_labels),4)
        Precision = round(precision_score(ground, pred_labels),4)
        F1_Score = round(f1_score(ground, pred_labels),4)
        kappa = round(cohen_kappa_score(ground,pred_labels),4)
        scores["Model"].append(pred_name)
        scores["Accuracy"].append(Accuracy)
        scores["Recall"].append(Recall)
        scores["Precision"].append(Precision)
        scores["F1_Score"].append(F1_Score)
        scores["Cohen's_Kappa"].append(kappa)


In [114]:
print("Alpaca Prompting", dataset_name)
print("----------------------------------------------------------------------------------")
df = pd.DataFrame(scores)
print(df.to_string(index=False))
print("----------------------------------------------------------------------------------")


Alpaca Prompting Vul_notPriorInCited
----------------------------------------------------------------------------------
                    Model  Accuracy  Recall  Precision  F1_Score  Cohen's_Kappa
                     DSV3    0.6618  0.1594     0.9565    0.2733         0.1798
                    GPT4o    0.6532  0.2319     0.6957    0.3478         0.1854
       unsloth-Qwen2.5-7B    0.7197  0.9855     0.5887    0.7371         0.4749
      unsloth-Qwen2.5-14B    0.7861  0.5362     0.8810    0.6667         0.5226
unsloth-Meta-Llama-3.1-8B    0.4364  0.6232     0.3755    0.4687        -0.0579
----------------------------------------------------------------------------------


In [131]:
print("Alpaca Prompting", dataset_name)
print("----------------------------------------------------------------------------------")
df = pd.DataFrame(scores)
print(df.to_string(index=False))
print("----------------------------------------------------------------------------------")


Alpaca Prompting Vul_notPriorInCited
----------------------------------------------------------------------------------
Model  Accuracy  Recall  Precision  F1_Score  Cohen's_Kappa
   P1    0.7434  0.9735     0.6652    0.7903         0.4884
   P2    0.7072  0.9768     0.6330    0.7682         0.4165
   P3    0.6118  0.9901     0.5620    0.7170         0.2275
----------------------------------------------------------------------------------


In [ ]:
pairwise_kappa = {}

kappas = []
for true_name, true_labels in mylabels.items():
    if true_name != "All": continue 
    for pred_name, pred_labels in labels.items():
        if len(true_labels) == len(pred_labels):  
            kappa = round(cohen_kappa_score(true_labels,pred_labels),4)
            kappas.append(kappa)
            if "DS" in pred_name: 
                other_pred_labels = labels['labels_all_GPT']
                kappa = round(cohen_kappa_score(pred_labels,other_pred_labels),4)
                kappas.append(kappa) 
float(round(sum(kappas)/len(kappas),4))

In [9]:
verses = {'Vulgate_PriorNotInCited':verses_VnotPinC,
            'pre-Elizabethan_qp':verses_preE,
            'All':verses_all}

for name, verse_set in verses.items():
    for idx, id_string in enumerate(verse_set): 
        version, part, book = get_bible_info(id_string)
        verses[name][idx] = (version,part,book)

In [ ]:
# 0 - Version, 1 - Part, 2 - Book
fidx_f = {0:'Version',1:'Part',2:'Book'}
all_df = pd.DataFrame(columns=['Feature_Type','Feature','Model','Accuracy','Precision','F1_Score'])

for fidx in [1,0,2]: 
    print(fidx_f[fidx])
    for true_name, true_labels in mylabels.items(): 
        if "All" != true_name: continue 
        flist = [f[fidx] for f in verses[true_name]]
        unique = list(set(flist))
        specifics = {f:{
                    "Model": [],
                    "Accuracy": [],
                    "Precision": [],
                    "F1_Score": [],
                    "Cohen's Kappa":[]
                    } for f in unique} 
        for f in unique:
            for pred_name, pred_labels in labels.items(): 
                if len(true_labels) != len(pred_labels): continue 
                pred_name = pred_name.split("_")[-1]
                indices = []
                for i, item in enumerate(flist): 
                    if item == f: indices.append(i)
                y_t = [true_labels[i] for i in indices]
                y_p = [pred_labels[i] for i in indices]
                Accuracy = round(accuracy_score(y_t, y_p),4)
                Precision = round(precision_score(y_t, y_p),4)
                F1_Score = round(f1_score(y_t, y_p),4)
                kappa = round(cohen_kappa_score(y_t,y_p),4)
                specifics[f]["Model"].append(pred_name)
                specifics[f]["Accuracy"].append(Accuracy)
                specifics[f]["Precision"].append(Precision)
                specifics[f]["F1_Score"].append(F1_Score)
                specifics[f]["Cohen's Kappa"].append(kappa)
    for name,data in specifics.items(): 
        data['Feature'] = name
        data['Feature_Type'] = fidx_f[fidx]
        df = pd.DataFrame(data)
        df = df.rename(columns = {'Dataset_Model':'Model'})
        all_df = pd.concat([all_df,df],ignore_index=True)
all_df.to_csv(f"../assets/QP_labels/model_comparison.csv")

In [280]:
all_df

,Feature_Type,Feature,Model,Accuracy,Precision,F1_Score,Cohen's Kappa
0,Part,Old Testament,DS,0.8438,0.9333,0.8710,0.6753
1,Part,Old Testament,GPT,0.7838,0.9425,0.8089,0.5713
2,Part,New Testament,DS,0.8822,0.9538,0.9240,0.6639
3,Part,New Testament,GPT,0.7896,0.9706,0.8522,0.5052
4,Part,Apocrypha,DS,0.7286,0.9643,0.7397,0.4865
...,...,...,...,...,...,...,...
145,Book,Ecclesiastes,GPT,0.7778,1.0000,0.8333,0.5263
146,Book,Joshua,DS,1.0000,0.0000,0.0000,NaN
147,Book,Joshua,GPT,0.9000,0.0000,0.0000,0.0000
148,Book,1 Maccabees,DS,0.4286,0.6667,0.3333,0.0175


# Identify mis-matches between myself & V3 for fine-tuning 

In [ ]:
mismatches = {}

# target_ground = ground_preE
target_ground = VnotPinC

for idx, label in enumerate(ground_preE): 
    V3_label = labels_preE_DS[idx]
    if label != V3_label: 
        mismatches[idx] = (label,V3_label)

mismatch_set = []
targets = target_ground.to_dict(orient='records')
for idx, entry in enumerate(targets): 
    version, part, book = get_bible_info(entry['verse_id'])
    if idx in mismatches: 
        entry['label'] = mismatches[idx][0]
        entry['V3_Label'] = mismatches[idx][1] 
        entry['index'] = idx 
        entry['Version'] = version
        entry['Part'] = part 
        mismatch_set.append(entry)
mismatch_set = pd.DataFrame(mismatch_set)

In [18]:
mismatch_set.to_csv("../assets/QP_labels/mismatches_preE.csv")

In [20]:
mismatch_set.to_csv("../assets/QP_labels/mismatches_VnotPinC.csv")